# GRPO强化学习

In [1]:
from datasets import load_dataset
from trl import GRPOTrainer,GRPOConfig
import warnings
warnings.filterwarnings("ignore")

E:\web\LLM-tuning\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\web\LLM-tuning\.venv\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## 1、加载数据集

In [2]:
dataset = load_dataset("meta-math/GSM8K_zh", split="train[:1000]",cache_dir='../../datasets').select_columns(column_names=['question_zh','answer_zh','answer_only'])

'[WinError 10054] 远程主机强迫关闭了一个现有的连接。' thrown while requesting HEAD https://huggingface.co/datasets/meta-math/GSM8K_zh/resolve/4a5009abc37cbb2d3fd1a745f80e5ea1405ba9aa/GSM8K_zh.py
Retrying in 1s [Retry 1/5].
Using the latest cached version of the dataset since meta-math/GSM8K_zh couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at ..\..\datasets\meta-math___gsm8_k_zh\default\0.0.0\4a5009abc37cbb2d3fd1a745f80e5ea1405ba9aa (last modified on Thu Feb 26 15:22:05 2026).


In [3]:
dataset

Dataset({
    features: ['question_zh', 'answer_zh', 'answer_only'],
    num_rows: 1000
})

## 2、数据预处理

In [4]:
def covert_data(example):
    system_prompt='请以以下格式作答：\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>'
    return {'prompt':[{'role':'system','content':system_prompt},{'role':'user','content':example['question_zh']}],'answer':example['answer_only']}

In [5]:
dataset=dataset.map(covert_data,remove_columns=dataset.column_names)

In [6]:
dataset[0]

{'prompt': [{'content': '请以以下格式作答：\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>',
   'role': 'system'},
  {'content': 'Natalia在四月份向她的48个朋友出售了夹子，然后在五月份卖出了四月份的一半。Natalia在四月和五月总共卖了多少个夹子？',
   'role': 'user'}],
 'answer': '72'}

## 3、定义奖励函数

In [7]:
def extract_answer(text):
    answer=text.split('<answer>')[-1]
    answer=answer.split('</answer>')[0]
    return answer.strip()
def accuracy_reward(prompts,completions,answer,**kwargs):
    question=prompts[0][1]["content"]
    responses=[completion[0]['content'] for completion in completions]
    model_answer=[extract_answer(response) for response in responses]
    # print(f'问题：{question}\n答案：{answer[0]}\n模型回答：{responses[0]}\n模型答案：{model_answer[0]}')
    return [2.0 if r==a else 0.0 for r,a in zip(model_answer,answer)]

## 3、配置GRPOConfig

In [8]:
args=GRPOConfig(
    output_dir='../../models/grpo_model',
    learning_rate=5e-6,
    per_device_train_batch_size=2,
    num_generations=2,
    num_train_epochs=1,
)

In [9]:
trainer = GRPOTrainer(
    model="Qwen/Qwen3-0.6B",
    reward_funcs=accuracy_reward,
    train_dataset=dataset,
)

Loading weights: 100%|██████████| 311/311 [00:00<00:00, 485.03it/s, Materializing param=model.norm.weight]                              
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [10]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
Passing `generation_config` together with generation-related arguments=({'disable_compile'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


Step,Training Loss
10,-0.000695
20,0.009808
30,0.001458
40,0.000396
50,0.014061
60,-0.001873
70,0.014110
80,0.021319
90,0.018699
100,0.014187



KeyboardInterrupt

